In [3]:
from transformers import AutoModel, AutoProcessor
import torch
import torchaudio

# load audio
wav, sr = torchaudio.load("audio.wav")
# resample if necessary
wav = torchaudio.functional.resample(wav, sr, 16000)

# load model and processor
processor = AutoProcessor.from_pretrained("waveletdeboshir/gigaam-rnnt", trust_remote_code=True)
model = AutoModel.from_pretrained("waveletdeboshir/gigaam-rnnt", trust_remote_code=True)
model.eval()

input_features = processor(wav[0], sampling_rate=16000, return_tensors="pt")

# greedy prediction
with torch.no_grad():
    pred_ids = model.generate(**input_features)

# decode token ids to text
transcription = processor.batch_decode(pred_ids)[0]


You are using a model of type gigaam-rnnt to instantiate a model of type gigaam. This is not supported for all configurations of models and can yield errors.


In [4]:
def get_model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    size_all_mb = (param_size + buffer_size) / 1024**2
    print(f"Размер модели (вычислено): {size_all_mb:.2f} MB")
    return size_all_mb


In [5]:
get_model_size(model=model)

Размер модели (вычислено): 893.62 MB


893.6179275512695

In [10]:
import torch
import torchaudio
from transformers import AutoProcessor, AutoModel
import time
import psutil
from datasets import load_dataset, Audio
import evaluate # Для метрик WER и CER
import os
import gc
from tqdm.auto import tqdm

# --- Конфигурация ---
MODEL_ID = "waveletdeboshir/gigaam-rnnt" # <--- ИЗМЕНЕНО: ID новой модели
DATASET_ID = "mozilla-foundation/common_voice_16_1"
DATASET_NAME = "ru"
DATASET_SPLIT = "test"
NUM_SAMPLES_FOR_QUALITY_TEST = 100
TARGET_SAMPLE_RATE = 16000

# --- 1. Загрузка модели и процессора ---
print(f"Загрузка процессора {MODEL_ID}...")
processor = AutoProcessor.from_pretrained(MODEL_ID, trust_remote_code=True)
print(f"Загрузка модели {MODEL_ID}...")
model = AutoModel.from_pretrained(MODEL_ID, trust_remote_code=True)

# --- 2. Определение устройства и перемещение модели ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval() # Переводим модель в режим оценки
print(f"Модель перемещена на устройство: {device}")

# --- 3. Загрузка метрик WER и CER ---
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")
print("Метрики WER и CER загружены.")

# --- 4. Загрузка и подготовка датасета ---
print(f"Загрузка датасета {DATASET_ID}, конфигурация: {DATASET_NAME}, сплит: {DATASET_SPLIT}...")
dataset = load_dataset(DATASET_ID, DATASET_NAME, split=f"{DATASET_SPLIT}[:{NUM_SAMPLES_FOR_QUALITY_TEST}]", trust_remote_code=True)
print(f"Загружено {len(dataset)} сэмплов для оценки.")
dataset = dataset.cast_column("audio", Audio(sampling_rate=TARGET_SAMPLE_RATE))
print(f"Аудио в датасете преобразовано к {TARGET_SAMPLE_RATE} Hz.")

# --- 5. Функция для инференса и декодирования (АДАПТИРОВАНА ДЛЯ RNN-T) ---
def transcribe_audio(batch):
    audio_input = batch["audio"]
    raw_audio = audio_input["array"]
    sampling_rate = audio_input["sampling_rate"]

    if len(raw_audio) == 0:
        return {"reference_transcription": batch["sentence"], "predicted_transcription": ""}

    waveform_tensor = torch.tensor(raw_audio).unsqueeze(0)
    
    # Входные данные для RNN-T модели готовятся процессором и сразу перемещаются на нужное устройство
    input_features = processor(waveform_tensor[0], sampling_rate=sampling_rate, return_tensors="pt").to(device)
    
    with torch.no_grad():
        # Используем model.generate() для RNN-T моделей
        # input_features уже на нужном устройстве (device)
        pred_ids = model.generate(**input_features) 
    
    # Декодируем ID токенов в текст
    # ВАЖНО: используем skip_special_tokens=True, чтобы убрать специальные токены (например, <pad>, <s>, </s>)
    # pred_ids обычно имеет форму (batch_size, sequence_length), для одного сэмпла (1, sequence_length)
    transcription = processor.batch_decode(pred_ids, skip_special_tokens=True)[0]
    
    return {"reference_transcription": batch["sentence"], "predicted_transcription": transcription.strip()} # Добавил .strip() для удаления возможных пробелов по краям

# --- 6. Оценка качества на датасете ---
predictions = []
references = []

print(f"\nНачало оценки качества на {NUM_SAMPLES_FOR_QUALITY_TEST} сэмплах для {MODEL_ID}...")
for i in tqdm(range(len(dataset)), desc="Оценка качества"):
    sample = dataset[i]
    
    if not sample["sentence"] or sample["sentence"].strip() == "":
        continue

    try:
        result = transcribe_audio(sample)
        
        predictions.append(result["predicted_transcription"].lower())
        references.append(result["reference_transcription"].lower())

    except Exception as e:
        print(f"Ошибка при обработке сэмпла {i} ({sample.get('path', 'N/A')}): {e}")
        # Можно добавить пустые строки или пропустить, чтобы не сломать расчет метрики
        # predictions.append("") 
        # references.append(sample["sentence"].lower() if sample["sentence"] else "ошибка_при_обработке")


    if device.type == 'cuda':
        torch.cuda.empty_cache()
    gc.collect()

# --- 7. Расчет и вывод метрик WER и CER ---
if predictions and references:
    # Перед расчетом метрик убедимся, что у нас одинаковое количество предсказаний и эталонов
    if len(predictions) != len(references):
        print(f"Внимание: Количество предсказаний ({len(predictions)}) не совпадает с количеством эталонов ({len(references)}). Проверьте наличие ошибок.")
        # Можно попытаться выровнять, но лучше разобраться в причине
        min_len = min(len(predictions), len(references))
        predictions = predictions[:min_len]
        references = references[:min_len]

    if not predictions: # Если после фильтрации ничего не осталось
        print("\nНе удалось собрать предсказания для расчета WER/CER.")
    else:
        wer_score = wer_metric.compute(predictions=predictions, references=references)
        cer_score = cer_metric.compute(predictions=predictions, references=references)
        print(f"\nКачество модели (Word Error Rate - WER) для {MODEL_ID}: {wer_score:.4f}")
        print(f"Качество модели (Character Error Rate - CER) для {MODEL_ID}: {cer_score:.4f}")
else:
    print("\nНе удалось собрать предсказания и/или эталоны для расчета WER/CER.")

print("\nПримеры транскрипций:")
for i in range(min(5, len(predictions))): # Убедимся, что predictions не пустой
    print(f"Эталон  : {references[i] if i < len(references) else 'N/A'}")
    print(f"Предсказание: {predictions[i] if i < len(predictions) else 'N/A'}\n")

Загрузка процессора waveletdeboshir/gigaam-rnnt...
Загрузка модели waveletdeboshir/gigaam-rnnt...


You are using a model of type gigaam-rnnt to instantiate a model of type gigaam. This is not supported for all configurations of models and can yield errors.


Модель перемещена на устройство: cuda
Метрики WER и CER загружены.
Загрузка датасета mozilla-foundation/common_voice_16_1, конфигурация: ru, сплит: test...
Загружено 100 сэмплов для оценки.
Аудио в датасете преобразовано к 16000 Hz.

Начало оценки качества на 100 сэмплах для waveletdeboshir/gigaam-rnnt...


Оценка качества: 100%|██████████| 100/100 [00:27<00:00,  3.66it/s]


Качество модели (Word Error Rate - WER) для waveletdeboshir/gigaam-rnnt: 0.3041
Качество модели (Character Error Rate - CER) для waveletdeboshir/gigaam-rnnt: 0.0882

Примеры транскрипций:
Эталон  : масштабы финансово-экономического кризиса и темпы его распространения застали самых опытных специалистов мира врасплох.
Предсказание: масштабы финансово экономического кризиса и темпы его распространения застали самых опытных специалистов мира врасплох

Эталон  : к сожалению, эти предложения не нашли отражения в тексте.
Предсказание: к сожалению эти предложения не нашли отражения в тексте

Эталон  : мы настоятельно призываем посла танина незамедлительно провести неофициальное пленарное заседание для обсуждения реформы совета.
Предсказание: мы настоятельно призываем посла танина незамедлительно провести неофициальное пленарное заседание для обсуждения реформы совета

Эталон  : толпа озвереет, будет тереться, ощетинит ножки стоглавая вошь.
Предсказание: толпа зверет будет тереться ощетинит но

In [12]:
import torch
import torchaudio
from transformers import AutoProcessor, AutoModel
import time
import psutil
import os
import gc
from datasets import load_dataset, Audio

# --- Конфигурация ---
MODEL_ID = "waveletdeboshir/gigaam-rnnt" # <--- ИЗМЕНЕНО: ID новой модели
DATASET_ID = "mozilla-foundation/common_voice_16_1"
DATASET_NAME = "ru"
DATASET_SPLIT = "test"
NUM_SAMPLES_FOR_TIME_TEST = 20
TARGET_SAMPLE_RATE = 16000
NUM_WARMUP_RUNS = 5

# --- 1. Загрузка модели и процессора ---
print(f"Загрузка процессора {MODEL_ID}...")
processor = AutoProcessor.from_pretrained(MODEL_ID, trust_remote_code=True)
print(f"Загрузка модели {MODEL_ID}...")
model = AutoModel.from_pretrained(MODEL_ID, trust_remote_code=True)
model.eval() 
print("Модель и процессор загружены.")

# --- 2. Подготовка сэмплов ---
print(f"Подготовка {NUM_SAMPLES_FOR_TIME_TEST + NUM_WARMUP_RUNS} сэмплов для замеров...")
dataset_for_timing = load_dataset(
    DATASET_ID, 
    DATASET_NAME, 
    split=f"{DATASET_SPLIT}[:{NUM_SAMPLES_FOR_TIME_TEST + NUM_WARMUP_RUNS}]",
    trust_remote_code=True
)
dataset_for_timing = dataset_for_timing.cast_column("audio", Audio(sampling_rate=TARGET_SAMPLE_RATE))

# Сохраняем уже обработанные процессором данные (на CPU)
processed_inputs_list = []
for i in range(NUM_SAMPLES_FOR_TIME_TEST + NUM_WARMUP_RUNS):
    sample_audio = dataset_for_timing[i]["audio"]["array"]
    waveform_tensor = torch.tensor(sample_audio).unsqueeze(0) # (1, num_samples)
    # Процессор вызывается один раз для каждого сэмпла
    input_features = processor(waveform_tensor[0], sampling_rate=TARGET_SAMPLE_RATE, return_tensors="pt")
    processed_inputs_list.append(input_features)
print("Сэмплы подготовлены.")

# --- Переменные для результатов ---
time_gpu_ms_avg = "N/A"
vram_usage_mb = "N/A"

# --- 3. Измерение на GPU (если доступно) ---
if torch.cuda.is_available():
    print("\n--- Измерение на GPU ---")
    device_gpu = torch.device("cuda")
    model.to(device_gpu)
    
    print(f"Прогрев GPU ({NUM_WARMUP_RUNS} запусков)...")
    for i in range(NUM_WARMUP_RUNS):
        # Перемещаем конкретный input_features на GPU перед использованием
        warmup_input = {k: v.to(device_gpu) for k, v in processed_inputs_list[i].items()}
        with torch.no_grad():
            _ = model.generate(**warmup_input) # <--- ИЗМЕНЕНО: используем model.generate()
            
    torch.cuda.synchronize(device_gpu) # Убедимся, что прогрев завершен
    torch.cuda.reset_peak_memory_stats(device_gpu) # Сбрасываем счетчик пиковой памяти ПОСЛЕ прогрева
    
    gpu_times = []
    print(f"Замеры времени инференса GPU ({NUM_SAMPLES_FOR_TIME_TEST} запусков)...")
    # Память измеряется для первого реального замера, т.к. reset_peak_memory_stats был до цикла
    # Если нужно измерять для каждого, то reset и get нужно делать внутри цикла (но это обычно не требуется)
    
    # Первый замер для VRAM
    current_input_for_vram = {k: v.to(device_gpu) for k, v in processed_inputs_list[NUM_WARMUP_RUNS].items()}
    with torch.no_grad():
        _ = model.generate(**current_input_for_vram)
    torch.cuda.synchronize(device_gpu)
    vram_usage_mb = torch.cuda.max_memory_allocated(device_gpu) / (1024 * 1024)
    del current_input_for_vram # Освобождаем память от этого инпута
    if NUM_SAMPLES_FOR_TIME_TEST == 0: # Если только прогрев
         time_gpu_ms_avg = 0

    for i in range(NUM_WARMUP_RUNS, NUM_WARMUP_RUNS + NUM_SAMPLES_FOR_TIME_TEST):
        current_input = {k: v.to(device_gpu) for k, v in processed_inputs_list[i].items()}
        torch.cuda.synchronize(device_gpu) # Синхронизация перед стартом таймера
        start_time = time.perf_counter()
        with torch.no_grad():
            _ = model.generate(**current_input) # <--- ИЗМЕНЕНО: используем model.generate()
        torch.cuda.synchronize(device_gpu) # Синхронизация после завершения операций на GPU
        end_time = time.perf_counter()
        gpu_times.append((end_time - start_time) * 1000) # Время в миллисекундах
        
    if gpu_times:
        time_gpu_ms_avg = sum(gpu_times) / len(gpu_times)
    
    print(f"Среднее время инференса на GPU: {time_gpu_ms_avg if isinstance(time_gpu_ms_avg, str) else f'{time_gpu_ms_avg:.2f}'} ms")
    print(f"Пиковое использование VRAM (измерено на первом реальном запуске): {vram_usage_mb:.2f} MB")
    
    del warmup_input, current_input # Очистка переменных
    model.to("cpu") # Перемещаем модель обратно на CPU
    torch.cuda.empty_cache() # Очищаем кэш CUDA
    gc.collect()
else:
    print("CUDA (GPU) не доступна. Пропуск измерений на GPU.")

# --- 4. Измерение на CPU ---
print("\n--- Измерение на CPU ---")
device_cpu = torch.device("cpu")
model.to(device_cpu) # Убедимся, что модель на CPU

process = psutil.Process(os.getpid())
ram_before_cpu_warmup_mb = process.memory_info().rss / (1024 * 1024)

print(f"Прогрев CPU ({NUM_WARMUP_RUNS} запусков)...")
for i in range(NUM_WARMUP_RUNS):
    # processed_inputs_list уже на CPU
    warmup_input = processed_inputs_list[i] 
    with torch.no_grad():
        _ = model.generate(**warmup_input) # <--- ИЗМЕНЕНО: используем model.generate()

cpu_times = []
print(f"Замеры времени инференса CPU ({NUM_SAMPLES_FOR_TIME_TEST} запусков)...")
# Измеряем RAM после прогрева, перед основным циклом замеров
ram_after_cpu_warmup_mb = process.memory_info().rss / (1024 * 1024) 

for i in range(NUM_WARMUP_RUNS, NUM_WARMUP_RUNS + NUM_SAMPLES_FOR_TIME_TEST):
    current_input = processed_inputs_list[i]
    start_time = time.perf_counter()
    with torch.no_grad():
        _ = model.generate(**current_input) # <--- ИЗМЕНЕНО: используем model.generate()
    end_time = time.perf_counter()
    cpu_times.append((end_time - start_time) * 1000)

if cpu_times:
    time_cpu_ms_avg = sum(cpu_times) / len(cpu_times)
else:
    time_cpu_ms_avg = "N/A" # или 0, если NUM_SAMPLES_FOR_TIME_TEST = 0

# Финальное измерение RAM после всех операций
ram_usage_mb = process.memory_info().rss / (1024 * 1024) 

print(f"Среднее время инференса на CPU: {time_cpu_ms_avg if isinstance(time_cpu_ms_avg, str) else f'{time_cpu_ms_avg:.2f}'} ms")
print(f"Использование RAM (после инференсов на CPU): {ram_usage_mb:.2f} MB")
print(f"(RAM до прогрева CPU: {ram_before_cpu_warmup_mb:.2f} MB, после прогрева CPU: {ram_after_cpu_warmup_mb:.2f} MB)")


# --- 5. Итоговые результаты для таблицы ---
print("\n--- Итоговые результаты для таблицы ---")
print(f"Модель: {MODEL_ID}")
print(f"Время инференса (CPU, ms): {time_cpu_ms_avg if isinstance(time_cpu_ms_avg, str) else f'{time_cpu_ms_avg:.2f}'}")
print(f"Время инференса (GPU, ms): {time_gpu_ms_avg if isinstance(time_gpu_ms_avg, str) else f'{time_gpu_ms_avg:.2f}'}")
print(f"Использование RAM (MB): {ram_usage_mb:.2f}") # Финальное значение
print(f"Использование VRAM (MB): {vram_usage_mb if isinstance(vram_usage_mb, str) else f'{vram_usage_mb:.2f}'}")

Загрузка процессора waveletdeboshir/gigaam-rnnt...
Загрузка модели waveletdeboshir/gigaam-rnnt...


You are using a model of type gigaam-rnnt to instantiate a model of type gigaam. This is not supported for all configurations of models and can yield errors.


Модель и процессор загружены.
Подготовка 25 сэмплов для замеров...
Сэмплы подготовлены.

--- Измерение на GPU ---
Прогрев GPU (5 запусков)...
Замеры времени инференса GPU (20 запусков)...
Среднее время инференса на GPU: 88.79 ms
Пиковое использование VRAM (измерено на первом реальном запуске): 2158.08 MB

--- Измерение на CPU ---
Прогрев CPU (5 запусков)...
Замеры времени инференса CPU (20 запусков)...
Среднее время инференса на CPU: 836.99 ms
Использование RAM (после инференсов на CPU): 3056.15 MB
(RAM до прогрева CPU: 3036.06 MB, после прогрева CPU: 3056.30 MB)

--- Итоговые результаты для таблицы ---
Модель: waveletdeboshir/gigaam-rnnt
Время инференса (CPU, ms): 836.99
Время инференса (GPU, ms): 88.79
Использование RAM (MB): 3056.15
Использование VRAM (MB): 2158.08
